In [1]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du
from sklearn.externals import joblib

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")
# df_ratio = pd.read_csv("Feature_matrix/df_new_has_all_the_ratio_201808031807.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
vn_and_weekdays_cols = ["VisitNumber", "TripType", 'Return', 'Monday', 'Tuesday',\
                        'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def get_pivor_df(df, col):
    """
        example) 
            df_train_cp = get_pivor_df(df_decoded, "Company") 이후에,
            df_train_cp = get_rid_of_zero_units(df_train_cp) 를 사용해서 음수를 없애줄것.
    """
    
    return pd.pivot_table(data= df, index="VisitNumber", fill_value=0,\
                          values="ScanCount", columns=col, aggfunc=np.sum)

def get_rid_of_zero_units(df):
    return pd.DataFrame(np.where(df < 0, 0, df), columns = df.columns)

In [3]:
def getSpecifiedVisitNumberData(df_train, vn):
    display(df_train[df_train.VisitNumber == vn])
#     display(df_train_dd[df_train_dd.VisitNumber == vn])

In [4]:
def get_real_category_cnt(df_, col = "DepartmentDescription", threshold=1.0):
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    
    # sc_count가 음수인 아이템은 같은 날 산 아이템이라 볼 수 없다.
    # 리턴하기위해 가져온 것들이므로 월마트를 나갈 때 들고가는 아이템이라 보지않아도된다. 
    # 그런 아이템은 모두 nan으로 바꾸고 drop한다.
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] <= 0, np.nan, df_count["Sc_sum"])
    df_count = df_count.dropna()
    
    # 남은 아이템들만 더해주면 위에서 지정해준 컬럼의 유니크한 개수만 남는다.
    df_count = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Sc_sum")
    
    return list(df_count[df_count.Sc_sum == threshold].VisitNumber.unique())

In [5]:
def get_flat_type_user(df_, col = "DepartmentDescription"):
    """
        예:) vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded) 반드시 df는 decoding 함수로 decoded된 것을 넣을 것.
    """
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] <= 0, np.nan, df_count["Sc_sum"])
    df_count = df_count.dropna()
    df_count_total = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Total")
    df_merged = pd.merge(df_count, df_count_total, on="VisitNumber")
    df_merged["P"] = df_merged["Sc_sum"].div(df_merged["Total"])
#     df_merged = df_merged[df_merged["Total"] != 1.0]
    
    one_item_only_vn = []
    more_than_one_vn_count = []
    for i, vn in enumerate(df_merged.VisitNumber.unique()):
        tmp_df = df_merged[df_merged["VisitNumber"] == vn]
        unique_p = list(tmp_df["P"].unique())
        length_unique_p = len(unique_p)
        if length_unique_p == 1:
            if unique_p[0] == 1:
                one_item_only_vn.append(vn)
            else:
                more_than_one_vn_count.append(vn)
        if i % 5000 == 0:
            print(str(i) + "까지 진행됨.")
    
    return more_than_one_vn_count, one_item_only_vn

In [6]:
# import decode_utils as du
# import awesome_functions as af
from sklearn.metrics import confusion_matrix
from IPython.display import display, Markdown

def getAccuracy(y_true, y_pred, data_length, i):
    """
        y_true : 원래 타겟 컬럼의 데이터를 넣어준다.
        y_pred : 예측한 값을 넣어준다.
        data_length : 예측한 데이터의 총 개수를 넣어준다.
    """
    accuracy = round(np.trace(confusion_matrix(y_true, y_pred))/data_length, 4)
    display(Markdown("##### Accuracy : " + str(accuracy) + ", smoothing : " + str(i)))
    return accuracy

def fitNaiveBayesModel_smoothing(X, y, a = 1.0):
    return MultinomialNB(alpha=a).fit(X, y)

def __combine_dd_fl(df_dd, df_fl):
    col_we_dont_need = ["VisitNumber", "TripType", 'Return', 'Monday',\
                        'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    cols_we_need = [col for col in df_dd.columns if col not in col_we_dont_need]
    cols = [col for col in cols_we_need if col != "VisitNumber"]
    return du.concatDf(df_dd[cols], df_fl)
    
def get_concat_df_for_test_df(df_dd, df_fl):
    return __combine_dd_fl(df_dd, df_fl)
    
def get_df_has_specified_vn_list(vn_list, df=None, df_dd=None, df_fl=None, is_in=True,
                                 is_need_to_make_df = False):
    if is_need_to_make_df:
        df = __combine_dd_fl(df_dd, df_fl)
        print("Concat 완료.")
    if is_in:
        return df[df.VisitNumber.isin(vn_list)]
    return df[~df.VisitNumber.isin(vn_list)]

def test_multinomial(df, early_stop = 5):
    X, y = af.get_df_to_fit(df)
    a = np.arange(0, 0.16, 0.01)
    accuracy_ = 0
    count = 0
    result = []
    alpha = []
    for i in a:
        if count >= early_stop:
            best_score = np.array(result).max()
            idx = result.index(best_score)
            display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
            return
        model_nb = fitNaiveBayesModel_smoothing(X, y, i)
        y_pred = model_nb.predict(X)
        accuracy = getAccuracy(y, y_pred, len(X), i)
        if accuracy_ > accuracy:
            count += 1
        accuracy_ = accuracy
        result.append(accuracy)
        alpha.append(i)
    best_score = np.array(result).max()
    idx = result.index(best_score)
    display(Markdown("## Best_score : " + str(best_score) + ", Alpha : " + str(alpha[idx])))
#     return best_score, alpha[idx]

def cal_average_of_accuracy(li1, li2, score_li):    
    li3 = 96574 - len(li1 + li2)
    weight = [len(li1), len(li2), li3]
    result = 0
    for i, score in enumerate(score_li):
        result += weight[i] * score
    result = result/96574
    print(result)
    return result

In [7]:
def saveModelObjectAsPickle(model, feature):
    filename = "feature_"+ str(feature) + "_xgb_trained_model.pkl"
    joblib.dump(model, filename)

In [8]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [9]:
df_test_decoded = du.decodeStuffNeedsToBeDecoded(df_test)

In [10]:
# vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded)

In [10]:
vn_fl_more_than_one, vn_fl_one = get_flat_type_user(df_decoded, "FinelineNumber")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.


In [11]:
vn_test_fl_more_than_one, vn_test_fl_one = get_flat_type_user(df_test_decoded, "FinelineNumber")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.


In [13]:
# vn_company_more_than_one, vn_company_one = get_flat_type_user(df_decoded, "Company")

In [12]:
cols = [col for col in df_train_dd.columns if col != "TripType"]

In [13]:
df_test_dd = df_test_dd[cols]

In [14]:
cols_fl = [col for col in df_train_fl.columns if col != "TripType"]

In [15]:
df_test_fl = df_test_fl[cols_fl]

# 현 최고모델 Classification report를 통해 Insight를 얻자

In [55]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_one, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [56]:
X_1_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [57]:
X_1_test.drop("index", axis=1, inplace = True)

In [119]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_more_than_one, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [120]:
X_2_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [103]:
X_2_test.columns == X_2.columns

array([ True,  True,  True, ...,  True,  True,  True])

In [121]:
X_2_test.drop("index", axis=1, inplace = True)

In [65]:
vn_test_fl_3 = vn_test_fl_one + vn_test_fl_more_than_one

In [122]:
df_to_test = get_df_has_specified_vn_list(vn_test_fl_3, is_in=False, df_dd = df_test_dd, df_fl = df_test_fl, is_need_to_make_df = True)

Concat 완료.


In [123]:
X_3_test = af.get_df_to_fit(df_to_test, is_test_df=True)

In [124]:
X_3_test.drop("index", axis=1, inplace = True)

In [125]:
len(X_1_test), len(X_2_test), len(X_3_test)

(19313, 31216, 45145)

In [126]:
len(X_1_test) + len(X_2_test) + len(X_3_test)

95674

## feature 1 

In [16]:
df_to_test = get_df_has_specified_vn_list(vn_fl_one, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [17]:
X_1, y_1 = af.get_df_to_fit(df_to_test)

In [20]:
from scipy.sparse import csr_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(y_1)
y_labeled = label_enc.transform(y_1)

In [21]:
train_X, test_X, train_y, test_y = train_test_split(X_1, y_labeled, random_state=0)

In [22]:
csr_train = csr_matrix(train_X.values)
csr_test = csr_matrix(test_X.values)

In [23]:
if xgboost:
    dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
    dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
    num_boost_round = 500
    params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class':38,
              'max_depth':4,
              'eta': 0.25}

    evals = [(dtrain, 'train'), (dtest, 'eval')]

    bst = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:1.66166	eval-mlogloss:1.66334
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:1.31602	eval-mlogloss:1.3258
[2]	train-mlogloss:1.1202	eval-mlogloss:1.13234
[3]	train-mlogloss:0.992876	eval-mlogloss:1.00779
[4]	train-mlogloss:0.903931	eval-mlogloss:0.922093
[5]	train-mlogloss:0.832281	eval-mlogloss:0.852689
[6]	train-mlogloss:0.775618	eval-mlogloss:0.798166
[7]	train-mlogloss:0.728232	eval-mlogloss:0.752298
[8]	train-mlogloss:0.689028	eval-mlogloss:0.715336
[9]	train-mlogloss:0.656369	eval-mlogloss:0.683298
[10]	train-mlogloss:0.62861	eval-mlogloss:0.656622
[11]	train-mlogloss:0.604012	eval-mlogloss:0.633159
[12]	train-mlogloss:0.583449	eval-mlogloss:0.613185
[13]	train-mlogloss:0.564949	eval-mlogloss:0.594838
[14]	train-mlogloss:0.548756	eval-mlogloss:0.579191
[15]	train-mlogloss:0.534407	eval-mlogloss:0.565584
[16]	train-mlogloss:0.521388	eval-mlo

[155]	train-mlogloss:0.282513	eval-mlogloss:0.376065
[156]	train-mlogloss:0.282089	eval-mlogloss:0.375853
[157]	train-mlogloss:0.281656	eval-mlogloss:0.375643
[158]	train-mlogloss:0.281234	eval-mlogloss:0.375468
[159]	train-mlogloss:0.280796	eval-mlogloss:0.375314
[160]	train-mlogloss:0.280375	eval-mlogloss:0.375131
[161]	train-mlogloss:0.279923	eval-mlogloss:0.374978
[162]	train-mlogloss:0.279325	eval-mlogloss:0.374806
[163]	train-mlogloss:0.278959	eval-mlogloss:0.374763
[164]	train-mlogloss:0.278579	eval-mlogloss:0.374669
[165]	train-mlogloss:0.278211	eval-mlogloss:0.374417
[166]	train-mlogloss:0.277872	eval-mlogloss:0.374307
[167]	train-mlogloss:0.277529	eval-mlogloss:0.374098
[168]	train-mlogloss:0.27718	eval-mlogloss:0.374027
[169]	train-mlogloss:0.2768	eval-mlogloss:0.373897
[170]	train-mlogloss:0.276393	eval-mlogloss:0.373721
[171]	train-mlogloss:0.276012	eval-mlogloss:0.37376
[172]	train-mlogloss:0.275659	eval-mlogloss:0.373514
[173]	train-mlogloss:0.27524	eval-mlogloss:0.37338

# feature 2

In [30]:
df_to_test = get_df_has_specified_vn_list(vn_fl_more_than_one, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df=True)

Concat 완료.


In [31]:
X_2, y_2 = af.get_df_to_fit(df_to_test)

In [44]:
label_enc = LabelEncoder().fit(y_2)
y_labeled = label_enc.transform(y_2)

In [45]:
train_X, test_X, train_y, test_y = train_test_split(X_2, y_labeled, random_state=0)

In [46]:
csr_train = csr_matrix(train_X.values)
csr_test = csr_matrix(test_X.values)

In [55]:
if xgboost:
    dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
    dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
    num_boost_round = 500
    params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class':38,
              'max_depth':3,
              'eta': 0.25}

    evals = [(dtrain, 'train'), (dtest, 'eval')]

    bst_2 = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:2.45352	eval-mlogloss:2.463
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.20739	eval-mlogloss:2.22532
[2]	train-mlogloss:2.03683	eval-mlogloss:2.06199
[3]	train-mlogloss:1.90914	eval-mlogloss:1.94012
[4]	train-mlogloss:1.8097	eval-mlogloss:1.84626
[5]	train-mlogloss:1.7268	eval-mlogloss:1.76735
[6]	train-mlogloss:1.65856	eval-mlogloss:1.70286
[7]	train-mlogloss:1.60111	eval-mlogloss:1.64874
[8]	train-mlogloss:1.55113	eval-mlogloss:1.60206
[9]	train-mlogloss:1.50773	eval-mlogloss:1.56201
[10]	train-mlogloss:1.46929	eval-mlogloss:1.52746
[11]	train-mlogloss:1.43496	eval-mlogloss:1.49665
[12]	train-mlogloss:1.40467	eval-mlogloss:1.46952
[13]	train-mlogloss:1.37719	eval-mlogloss:1.44449
[14]	train-mlogloss:1.35302	eval-mlogloss:1.42287
[15]	train-mlogloss:1.33069	eval-mlogloss:1.40363
[16]	train-mlogloss:1.31038	eval-mlogloss:1.38645
[17]	train-ml

[159]	train-mlogloss:0.720036	eval-mlogloss:1.02329
[160]	train-mlogloss:0.718476	eval-mlogloss:1.02314
[161]	train-mlogloss:0.716973	eval-mlogloss:1.02233
[162]	train-mlogloss:0.715489	eval-mlogloss:1.02192
[163]	train-mlogloss:0.714032	eval-mlogloss:1.02137
[164]	train-mlogloss:0.712611	eval-mlogloss:1.02083
[165]	train-mlogloss:0.711085	eval-mlogloss:1.02029
[166]	train-mlogloss:0.709592	eval-mlogloss:1.02002
[167]	train-mlogloss:0.708009	eval-mlogloss:1.01939
[168]	train-mlogloss:0.706513	eval-mlogloss:1.01866
[169]	train-mlogloss:0.705125	eval-mlogloss:1.01844
[170]	train-mlogloss:0.703778	eval-mlogloss:1.01801
[171]	train-mlogloss:0.702231	eval-mlogloss:1.0175
[172]	train-mlogloss:0.700849	eval-mlogloss:1.01701
[173]	train-mlogloss:0.699455	eval-mlogloss:1.0167
[174]	train-mlogloss:0.698064	eval-mlogloss:1.01604
[175]	train-mlogloss:0.696649	eval-mlogloss:1.01545
[176]	train-mlogloss:0.695186	eval-mlogloss:1.01499
[177]	train-mlogloss:0.693628	eval-mlogloss:1.0144
[178]	train-mlo

[316]	train-mlogloss:0.546744	eval-mlogloss:0.978411
[317]	train-mlogloss:0.545926	eval-mlogloss:0.9784
[318]	train-mlogloss:0.545024	eval-mlogloss:0.978092
[319]	train-mlogloss:0.544104	eval-mlogloss:0.978042
[320]	train-mlogloss:0.543275	eval-mlogloss:0.97784
[321]	train-mlogloss:0.542359	eval-mlogloss:0.977507
[322]	train-mlogloss:0.541586	eval-mlogloss:0.977389
[323]	train-mlogloss:0.540821	eval-mlogloss:0.977353
[324]	train-mlogloss:0.540098	eval-mlogloss:0.97729
[325]	train-mlogloss:0.539389	eval-mlogloss:0.977312
[326]	train-mlogloss:0.538595	eval-mlogloss:0.977221
[327]	train-mlogloss:0.537725	eval-mlogloss:0.977097
[328]	train-mlogloss:0.53681	eval-mlogloss:0.97685
[329]	train-mlogloss:0.536061	eval-mlogloss:0.97659
[330]	train-mlogloss:0.53525	eval-mlogloss:0.97641
[331]	train-mlogloss:0.534455	eval-mlogloss:0.97629
[332]	train-mlogloss:0.533642	eval-mlogloss:0.976181
[333]	train-mlogloss:0.532823	eval-mlogloss:0.975977
[334]	train-mlogloss:0.532	eval-mlogloss:0.9759
[335]	tr

[472]	train-mlogloss:0.443312	eval-mlogloss:0.965553
[473]	train-mlogloss:0.442726	eval-mlogloss:0.965726
[474]	train-mlogloss:0.44223	eval-mlogloss:0.96577
[475]	train-mlogloss:0.441622	eval-mlogloss:0.965784
[476]	train-mlogloss:0.441104	eval-mlogloss:0.965815
[477]	train-mlogloss:0.44047	eval-mlogloss:0.965792
[478]	train-mlogloss:0.439805	eval-mlogloss:0.96562
[479]	train-mlogloss:0.439234	eval-mlogloss:0.965539
[480]	train-mlogloss:0.438567	eval-mlogloss:0.965515
[481]	train-mlogloss:0.437979	eval-mlogloss:0.965495
Stopping. Best iteration:
[471]	train-mlogloss:0.443814	eval-mlogloss:0.96548



# feature 3

In [36]:
vn_fl_3 = vn_fl_one + vn_fl_more_than_one

In [37]:
df_to_test = get_df_has_specified_vn_list(vn_fl_3, is_in=False, df_dd = df_train_dd, df_fl = df_train_fl, is_need_to_make_df = True)

Concat 완료.


In [38]:
X_3, y_3 = af.get_df_to_fit(df_to_test)

In [48]:
label_enc = LabelEncoder().fit(y_3)
y_labeled = label_enc.transform(y_3)

In [49]:
train_X, test_X, train_y, test_y = train_test_split(X_3, y_labeled, random_state=0)

In [50]:
csr_train = csr_matrix(train_X.values)
csr_test = csr_matrix(test_X.values)

In [54]:
if xgboost:
    dtrain = xgb.DMatrix(csr_train, label=train_y, feature_names = train_X.columns)
    dtest = xgb.DMatrix(csr_test, label=test_y, feature_names = train_X.columns)
    num_boost_round = 500
    params = {'objective': 'multi:softprob', 
              'eval_metric': 'mlogloss',
              'num_class': 38,
              'max_depth': 3,
              'eta': 0.25}

    evals = [(dtrain, 'train'), (dtest, 'eval')]

    bst_3 = xgb.train(params=params,  
                    dtrain=dtrain, 
                    num_boost_round=num_boost_round, 
                    evals=evals,
                    early_stopping_rounds=10,)

[0]	train-mlogloss:2.45352	eval-mlogloss:2.463
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.20739	eval-mlogloss:2.22532
[2]	train-mlogloss:2.03683	eval-mlogloss:2.06199
[3]	train-mlogloss:1.90914	eval-mlogloss:1.94012
[4]	train-mlogloss:1.8097	eval-mlogloss:1.84626
[5]	train-mlogloss:1.7268	eval-mlogloss:1.76735
[6]	train-mlogloss:1.65856	eval-mlogloss:1.70286
[7]	train-mlogloss:1.60111	eval-mlogloss:1.64874
[8]	train-mlogloss:1.55113	eval-mlogloss:1.60206
[9]	train-mlogloss:1.50773	eval-mlogloss:1.56201
[10]	train-mlogloss:1.46929	eval-mlogloss:1.52746
[11]	train-mlogloss:1.43496	eval-mlogloss:1.49665
[12]	train-mlogloss:1.40467	eval-mlogloss:1.46952
[13]	train-mlogloss:1.37719	eval-mlogloss:1.44449
[14]	train-mlogloss:1.35302	eval-mlogloss:1.42287
[15]	train-mlogloss:1.33069	eval-mlogloss:1.40363
[16]	train-mlogloss:1.31038	eval-mlogloss:1.38645
[17]	train-ml

[159]	train-mlogloss:0.720036	eval-mlogloss:1.02329
[160]	train-mlogloss:0.718476	eval-mlogloss:1.02314
[161]	train-mlogloss:0.716973	eval-mlogloss:1.02233
[162]	train-mlogloss:0.715489	eval-mlogloss:1.02192
[163]	train-mlogloss:0.714032	eval-mlogloss:1.02137
[164]	train-mlogloss:0.712611	eval-mlogloss:1.02083
[165]	train-mlogloss:0.711085	eval-mlogloss:1.02029
[166]	train-mlogloss:0.709592	eval-mlogloss:1.02002
[167]	train-mlogloss:0.708009	eval-mlogloss:1.01939
[168]	train-mlogloss:0.706513	eval-mlogloss:1.01866
[169]	train-mlogloss:0.705125	eval-mlogloss:1.01844
[170]	train-mlogloss:0.703778	eval-mlogloss:1.01801
[171]	train-mlogloss:0.702231	eval-mlogloss:1.0175
[172]	train-mlogloss:0.700849	eval-mlogloss:1.01701
[173]	train-mlogloss:0.699455	eval-mlogloss:1.0167
[174]	train-mlogloss:0.698064	eval-mlogloss:1.01604
[175]	train-mlogloss:0.696649	eval-mlogloss:1.01545
[176]	train-mlogloss:0.695186	eval-mlogloss:1.01499
[177]	train-mlogloss:0.693628	eval-mlogloss:1.0144
[178]	train-mlo

[316]	train-mlogloss:0.546744	eval-mlogloss:0.978411
[317]	train-mlogloss:0.545926	eval-mlogloss:0.9784
[318]	train-mlogloss:0.545024	eval-mlogloss:0.978092
[319]	train-mlogloss:0.544104	eval-mlogloss:0.978042
[320]	train-mlogloss:0.543275	eval-mlogloss:0.97784
[321]	train-mlogloss:0.542359	eval-mlogloss:0.977507
[322]	train-mlogloss:0.541586	eval-mlogloss:0.977389
[323]	train-mlogloss:0.540821	eval-mlogloss:0.977353
[324]	train-mlogloss:0.540098	eval-mlogloss:0.97729
[325]	train-mlogloss:0.539389	eval-mlogloss:0.977312
[326]	train-mlogloss:0.538595	eval-mlogloss:0.977221
[327]	train-mlogloss:0.537725	eval-mlogloss:0.977097
[328]	train-mlogloss:0.53681	eval-mlogloss:0.97685
[329]	train-mlogloss:0.536061	eval-mlogloss:0.97659
[330]	train-mlogloss:0.53525	eval-mlogloss:0.97641
[331]	train-mlogloss:0.534455	eval-mlogloss:0.97629
[332]	train-mlogloss:0.533642	eval-mlogloss:0.976181
[333]	train-mlogloss:0.532823	eval-mlogloss:0.975977
[334]	train-mlogloss:0.532	eval-mlogloss:0.9759
[335]	tr

[472]	train-mlogloss:0.443312	eval-mlogloss:0.965553
[473]	train-mlogloss:0.442726	eval-mlogloss:0.965726
[474]	train-mlogloss:0.44223	eval-mlogloss:0.96577
[475]	train-mlogloss:0.441622	eval-mlogloss:0.965784
[476]	train-mlogloss:0.441104	eval-mlogloss:0.965815
[477]	train-mlogloss:0.44047	eval-mlogloss:0.965792
[478]	train-mlogloss:0.439805	eval-mlogloss:0.96562
[479]	train-mlogloss:0.439234	eval-mlogloss:0.965539
[480]	train-mlogloss:0.438567	eval-mlogloss:0.965515
[481]	train-mlogloss:0.437979	eval-mlogloss:0.965495
Stopping. Best iteration:
[471]	train-mlogloss:0.443814	eval-mlogloss:0.96548



In [56]:
cal_average_of_accuracy(vn_fl_one, vn_fl_more_than_one, [0.367298, 0.96, 0.96])

0.8401327193447511


0.8401327193447511

In [127]:
y_pred_test_proba_1 = model_feature_1.predict_proba(X_1_test)
y_pred_test_proba_2 = model_feature_2.predict_proba(X_2_test)
y_pred_test_proba_3 = model_feature_3.predict_proba(X_3_test)

In [303]:
y_pred_proba_1 = model_feature_1.predict_proba(X_1)
y_pred_proba_2 = model_feature_2.predict_proba(X_2)
y_pred_proba_3 = model_feature_3.predict_proba(X_3)

In [169]:
sub_test.set_index("VisitNumber", inplace=True)

In [130]:
def get_proba_and_set_submission(models, features, targets, vns, submission):
    sub = submission.copy()
    sub.set_index("VisitNumber", inplace=True)
    for idx, model in enumerate(models):
        y_pred = model.predict(features[idx])
        y_proba = model.predict_proba(features[idx])
        tt_li = []
        vn_li = vns[idx]
        y_pred_unique = targets[idx].unique()
        for i in range(len(y_pred_unique)):
            tmp = "TripType_" + str(sorted(y_pred_unique)[i])
            tt_li.append(tmp)
        sub.at[vn_li, tt_li] = y_proba
        print(str(idx + 1) + " feature 완료.")
    return sub

In [263]:
sub_test.set_index("VisitNumber", inplace=True)

In [228]:
vn_test_fl_dice = list(df_to_test.VisitNumber.unique())

In [229]:
vns_test = [vn_test_fl_one, vn_test_fl_more_than_one, vn_test_fl_dice]

In [257]:
sub_test = get_proba_and_set_submission(models, features, targets, vns, sub_test)

1 feature 완료.
2 feature 완료.
3 feature 완료.


In [215]:
tt_li = []
for i in range(len(y_1_2.unique())):
    tmp = "TripType_" + str(sorted(y_1_2.unique())[i])
    tt_li.append(tmp)
len(tt_li)

37

In [175]:
submit = submission.copy()

In [177]:
submit_1_test = pd.DataFrame(y_pred_test_proba_1, columns = tt_li)

In [178]:
submit_1_test["VisitNumber"] = vn_test_fl_one

In [179]:
submit_1_test["TripType_14"] = 0
submit_1_test["TripType_43"] = 0
submit_1_test["TripType_44"] = 0

In [180]:
submit_1_test = submit_1_test[submit.columns]

In [228]:
submit_1_2_test = pd.DataFrame(y_pred_test_proba_1_2, columns = tt_li)

In [229]:
submit_1_2_test["TripType_14"] = 0

In [230]:
len(vn_test_fl_1_2), len(submit_1_2_test)

(50529, 50529)

In [231]:
submit_1_2_test["VisitNumber"] = vn_test_fl_1_2

In [232]:
submit_1_2_test = submit_1_2_test[submit.columns]

In [182]:
submit_2_3_test = pd.DataFrame(y_pred_test_proba_2_3, columns = tt_li)

In [183]:
submit_2_3_test["VisitNumber"] = vn_test_fl_2_3

In [184]:
submit_2_3_test = submit_2_3_test[submit.columns]

In [185]:
submit_test_all_2_3 = pd.concat([submit_1_test, submit_2_3_test])

In [186]:
submit_test_all_2_3 = submit_test_all_2_3.sort_values("VisitNumber")
submit_test_all_2_3.reset_index(drop=True, inplace=True)

In [189]:
af.saveDataFrameToCsv(submit_test_all_2_3, "last_try")

In [233]:
submit_test_all_1_2 = pd.concat([submit_1_2_test, submit_3_test])

In [234]:
submit_test_all_1_2 = submit_test_all_1_2.sort_values("VisitNumber")
submit_test_all_1_2.reset_index(drop=True, inplace=True)

In [236]:
af.saveDataFrameToCsv(submit_test_all_1_2, "get_insight")

In [190]:
df_to_test = get_concat_df_for_test_df(df_train_dd, df_train_fl)

In [192]:
X_1.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,1627.0,-1,8191.0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [193]:
X_test, y_test = af.get_df_to_fit(df_to_test)

In [194]:
X_test.head()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,1627.0,-1,8191.0,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [195]:
y_pred = model_feature_1.predict(X_test)

In [196]:
af.getAccuracy(y_test, y_pred, len(X_test))

##### Accuracy : 0.4634

In [136]:
submit_2_test = pd.DataFrame(y_pred_test_proba_2, columns = tt_li)

In [137]:
submit_2_test["VisitNumber"] = vn_test_fl_more_than_one

In [138]:
submit_2_test["TripType_14"] = 0

In [139]:
submit_2_test = submit_2_test[submit.columns]

In [140]:
vn_test_fl_dice = list(df_to_test.VisitNumber.unique())

In [141]:
len(vn_test_fl_dice)

45145

In [143]:
submit_3_test = pd.DataFrame(y_pred_test_proba_3, columns = tt_li)

In [144]:
submit_3_test["VisitNumber"] = vn_test_fl_dice

In [145]:
submit_3_test = submit_3_test[submit.columns]

In [147]:
submit_test_all = pd.concat([submit_1_test, submit_2_test, submit_3_test])

In [148]:
submit_test_all = submit_test_all.sort_values("VisitNumber")
submit_test_all.reset_index(drop=True, inplace=True)

In [152]:
submit_test_all.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,1.106492e-50,9.653557e-54,5.636323e-16,9.078964e-24,2.812958e-04,1.979547e-02,1.340561e-15,6.397182e-43,0.000000e+00,...,1.127518e-16,2.911286e-13,0.968618,0.011305,4.639459e-13,7.853481e-35,2.124187e-25,6.522341e-25,2.901157e-23,2.432760e-23
1,2,3.650732e-48,9.982242e-07,1.871108e-11,2.094947e-29,1.702872e-31,3.806059e-28,2.689917e-18,4.188307e-30,8.873491e-57,...,2.052912e-22,2.514577e-03,0.473402,0.143742,8.879635e-03,1.624739e-19,2.273621e-11,3.045582e-08,3.913261e-13,1.550635e-31
2,3,1.210754e-03,1.512679e-03,1.532167e-02,5.597487e-03,2.832801e-02,3.347708e-02,2.256775e-02,1.899446e-03,3.817823e-05,...,2.203266e-02,1.146737e-02,0.018215,0.070369,1.989943e-02,3.978709e-03,1.161236e-02,7.972996e-03,6.121550e-03,5.751394e-01
3,4,1.785875e-04,7.326824e-05,3.609887e-05,1.085832e-04,6.762326e-05,2.699233e-05,7.892577e-01,7.691106e-08,0.000000e+00,...,1.774390e-05,6.892610e-06,0.000006,0.000012,2.456320e-07,9.968076e-08,4.435841e-06,0.000000e+00,0.000000e+00,1.955480e-03
4,6,1.210754e-03,1.512679e-03,1.532167e-02,5.597487e-03,2.832801e-02,3.347708e-02,2.256775e-02,1.899446e-03,3.817823e-05,...,2.203266e-02,1.146737e-02,0.018215,0.070369,1.989943e-02,3.978709e-03,1.161236e-02,7.972996e-03,6.121550e-03,5.751394e-01


In [153]:
submit_to_check.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,3.175828e-44,1.002715e-40,2.166095e-13,4.179372e-22,6.724353e-03,1.363274e-01,2.948011e-13,1.035182e-41,0.000000e+00,...,1.666031e-13,2.328031e-12,0.826118,0.030830,1.237613e-12,7.923613e-33,1.815581e-23,3.778327e-23,2.590057e-22,5.927163e-22
1,2,9.340914e-37,6.497091e-08,1.836845e-11,4.432472e-30,5.931267e-32,8.130657e-30,1.061700e-18,1.666864e-30,1.255418e-36,...,2.480656e-22,3.706382e-03,0.202063,0.026589,9.295111e-03,6.476055e-20,2.154178e-11,5.776768e-09,3.698196e-13,2.854383e-31
2,3,1.477099e-03,1.905515e-03,1.947489e-02,7.086649e-03,3.607914e-02,3.982477e-02,2.684251e-02,2.461405e-03,5.035697e-05,...,2.841871e-02,1.510282e-02,0.023757,0.091342,2.647738e-02,5.176118e-03,1.514823e-02,1.023443e-02,8.074890e-03,4.568377e-01
3,4,1.785875e-04,7.326824e-05,3.609887e-05,1.085832e-04,6.762326e-05,2.699233e-05,7.892577e-01,7.691106e-08,0.000000e+00,...,1.774390e-05,6.892610e-06,0.000006,0.000012,2.456320e-07,9.968076e-08,4.435841e-06,0.000000e+00,0.000000e+00,1.955480e-03
4,6,1.477099e-03,1.905515e-03,1.947489e-02,7.086649e-03,3.607914e-02,3.982477e-02,2.684251e-02,2.461405e-03,5.035697e-05,...,2.841871e-02,1.510282e-02,0.023757,0.091342,2.647738e-02,5.176118e-03,1.514823e-02,1.023443e-02,8.074890e-03,4.568377e-01


In [151]:
af.saveDataFrameToCsv(submit_test_all, "submit_test_all_2")

In [97]:
submit_to_check = pd.read_csv("Feature_matrix/submit_please_get_great_score_201808080131.csv")

In [434]:
X, y = af.get_df_to_fit(df_train_fl)

In [436]:
model_test = fitNaiveBayesModel_smoothing(X, y, 0.0)

In [437]:
y_pred = model_test.predict(X)

In [438]:
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.6993

In [439]:
log_loss(y, model_test.predict_proba(X))

1.0689111273514562

In [429]:
af.saveDataFrameToCsv(submit_test_all, "submit_please_get_great_score")

In [337]:
submit_1 = pd.DataFrame(y_pred_proba_1, columns = tt_li)

In [338]:
submit_1["VisitNumber"] = vn_fl_one

In [339]:
submit_1["TripType_14"] = 0

In [340]:
submit_1["TripType_43"] = 0

In [341]:
submit_1["TripType_44"] = 0

In [331]:
sub_test.drop("TripType", axis =1 , inplace=True)

In [333]:
sub_test=sub_test.reset_index()

In [344]:
submit_1 = submit_1[sub_test.columns]

In [347]:
submit_2 = pd.DataFrame(y_pred_proba_2, columns = tt_li)

In [348]:
submit_2["TripType_14"] = 0

In [349]:
submit_2["VisitNumber"] = vn_fl_more_than_one

In [352]:
submit_2 = submit_2[sub_test.columns]

In [354]:
submit_3 = pd.DataFrame(y_pred_proba_3, columns = tt_li)

In [364]:
vn_fl_dice = list(df_to_test.VisitNumber.unique())

In [365]:
submit_3["VisitNumber"] = vn_fl_dice

In [367]:
submit_3 = submit_3[sub_test.columns]

In [371]:
submit_all = pd.concat([submit_1, submit_2, submit_3])

In [373]:
submit_all = submit_all.sort_values("VisitNumber")

In [377]:
submit_all.reset_index(drop=True, inplace=True)

In [379]:
submit_all.set_index("VisitNumber", inplace=True)

In [279]:
sub_test[sub_test.index.isin(vn_fl_one)].values[0]

array([1.41254359e-06, 5.79517666e-07, 2.85525253e-07, 6.99342921e-06,
       5.34868521e-07, 9.51288050e-01, 2.52301732e-07, 6.08330653e-10,
       7.52377882e-10, 9.56470393e-08, 1.09051498e-07, 5.89798632e-07,
       1.14668745e-07, 9.76530584e-08, 6.54228115e-07, 3.42084355e-07,
       3.67567079e-07, 1.02254938e-07, 1.43131129e-07, 1.08304044e-07,
       1.80553735e-07, 6.11786011e-08, 5.39045447e-07, 5.89872472e-07,
       7.51374501e-08, 5.87045639e-08, 6.14542455e-08, 4.84732605e-02,
       1.40345978e-07, 5.45173372e-08, 5.03687637e-08, 9.36357698e-08,
       1.94283506e-09, 7.88428341e-10, 3.50854360e-08, 5.71881452e-09,
       7.52377882e-10, 2.23956759e-04])

In [446]:
submit_test_all.set_index("VisitNumber", inplace=True)

In [230]:
submit_fl = get_proba_and_set_submission(models, features_test, targets, vns_test, submission)

1 feature 완료.
2 feature 완료.
3 feature 완료.


In [234]:
submit_fl = submit_fl.reset_index()

In [236]:
af.saveDataFrameToCsv(submit_fl, "submit_nb_tried_three_features")

In [574]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder().fit(y_1)
y_labeled = label_enc.transform(y_1)

In [575]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_labeled, random_state=0)

In [576]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [578]:
num_boost_round = 300
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 35, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

[0]	train-mlogloss:2.9518	eval-mlogloss:2.95404
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.41875	eval-mlogloss:2.4239
[2]	train-mlogloss:1.98923	eval-mlogloss:1.99831
[3]	train-mlogloss:1.66572	eval-mlogloss:1.67476
[4]	train-mlogloss:1.44663	eval-mlogloss:1.45736
[5]	train-mlogloss:1.28503	eval-mlogloss:1.30006
[6]	train-mlogloss:1.15639	eval-mlogloss:1.1728
[7]	train-mlogloss:1.05341	eval-mlogloss:1.07154
[8]	train-mlogloss:0.969075	eval-mlogloss:0.989787
[9]	train-mlogloss:0.898572	eval-mlogloss:0.921121
[10]	train-mlogloss:0.837392	eval-mlogloss:0.860966
[11]	train-mlogloss:0.786237	eval-mlogloss:0.811636
[12]	train-mlogloss:0.742256	eval-mlogloss:0.770075
[13]	train-mlogloss:0.70336	eval-mlogloss:0.731789
[14]	train-mlogloss:0.669246	eval-mlogloss:0.698751
[15]	train-mlogloss:0.640082	eval-mlogloss:0.670561
[16]	train-mlogloss:0.614212	eval-mlogloss:0.64

[155]	train-mlogloss:0.266862	eval-mlogloss:0.367576
[156]	train-mlogloss:0.266475	eval-mlogloss:0.367413
[157]	train-mlogloss:0.266077	eval-mlogloss:0.367257
[158]	train-mlogloss:0.265656	eval-mlogloss:0.367059
[159]	train-mlogloss:0.265275	eval-mlogloss:0.366842
[160]	train-mlogloss:0.26489	eval-mlogloss:0.366648
[161]	train-mlogloss:0.26444	eval-mlogloss:0.366539
[162]	train-mlogloss:0.264052	eval-mlogloss:0.366425
[163]	train-mlogloss:0.263638	eval-mlogloss:0.366241
[164]	train-mlogloss:0.263185	eval-mlogloss:0.366225
[165]	train-mlogloss:0.262759	eval-mlogloss:0.366192
[166]	train-mlogloss:0.262383	eval-mlogloss:0.366134
[167]	train-mlogloss:0.262007	eval-mlogloss:0.365964
[168]	train-mlogloss:0.261601	eval-mlogloss:0.365746
[169]	train-mlogloss:0.26112	eval-mlogloss:0.365508
[170]	train-mlogloss:0.260762	eval-mlogloss:0.365492
[171]	train-mlogloss:0.260425	eval-mlogloss:0.365353
[172]	train-mlogloss:0.260108	eval-mlogloss:0.365299
[173]	train-mlogloss:0.259741	eval-mlogloss:0.365

In [600]:
af.saveDataFrameToCsv(X_1, "X_1")
af.saveDataFrameToCsv(y_1, "y_1")
af.saveDataFrameToCsv(X_2, "X_2")
af.saveDataFrameToCsv(y_2, "y_2")
af.saveDataFrameToCsv(X_3, "X_3")
af.saveDataFrameToCsv(y_3, "y_3")

In [579]:
label_enc_2 = LabelEncoder().fit(y_2)
y_labeled = label_enc_2.transform(y_2)

In [580]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y_labeled, random_state=0)

In [581]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [ ]:
num_boost_round = 50
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 37, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst_2 = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)

In [ ]:
label_enc_3 = LabelEncoder().fit(y_3)
y_labeled = label_enc_3.transform(y_3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y_labeled, random_state=0)

In [ ]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [ ]:
num_boost_round = 50
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class': 38, 
          'max_delta_step': 3, 
          'eta': 0.2}

evals = [(dtrain, 'train'), (dtest, 'eval')]

bst_3 = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
                early_stopping_rounds=10,)